## 初始大模型调用函数

In [1]:
import json
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

# 初始化模型
llm = ChatOpenAI(
        model="deepseek-chat",
        temperature=0.5,
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url=os.getenv("DASHSCOPE_BASE_URL")
    )

def weather_api(location, date):
    # 模拟后端 API
    return {
        "location": location,
        "date": date,
        "forecast": "Sunny, 18°C"
    }


def ask_model_for_weather_params(question: str):
    prompt = f"""
            用户提问：{question}

            请只返回一个 JSON:
            {{
                "location": "...",
                "date": "..."
            }}
            不要输出任何解释、文字或额外字符！
            """
    resp = llm.invoke(prompt)
    text = resp.content.strip()
    print("模型原始输出：", text)

    try:
        data = json.loads(text)
    except:
        raise RuntimeError("模型输出的 JSON 非法，请手动修复或重试")

    return data


# 调用测试
question = "帮我查一下明天巴黎的天气"
params = ask_model_for_weather_params(question)
print("解析后的参数：", params)

result = weather_api(params["location"], params["date"])
print("最终结果：", result)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## OpenAI定义函数调用交互标准

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,ToolMessage
from dotenv import load_dotenv

load_dotenv()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取提供的地点当前天气.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "多云 12°C"
                    }
                },
                "required": ["location"],
                "additionalProperties": False
            }
        }
    }
]

llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.5,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
)

user_messages = [
    HumanMessage(content="帮我查一下当前巴黎的天气")
]

ai_msg = llm.invoke(user_messages, tools=tools)

print("模型输出:",ai_msg.content)
#迭代模型响应元数据
for key, value in ai_msg.response_metadata.items():
    print(f"{key}: {value}")



## LangChain底层封装调用工具

In [ ]:
import json
import os
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.messages import HumanMessage,  ToolMessage
from dotenv import load_dotenv


load_dotenv()

# ------------------
# 1. 定义工具
# ------------------
@tool
def get_weather(location: str, date: str):
    """获取指定地点日期的天气"""
    return {
        "location": location,
        "date": date,
        "forecast": "多云 12°C"
    }

# ------------------
# 2. 创建模型并绑定工具
# ------------------
llm = ChatOpenAI(
        model="deepseek-chat",
        temperature=0.5,
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url=os.getenv("DASHSCOPE_BASE_URL")
    ).bind_tools([get_weather])


# ------------------
# 3. 用户输入
# ------------------
user_msg = HumanMessage(content="帮我查一下2025年12月9日的巴黎的天气")

# ------------------
# 4. 模型第一轮：可能返回工具调用
# ------------------
ai_msg = llm.invoke([user_msg])

print("模型输出:",ai_msg.content)

#迭代模型响应元数据
for key, value in ai_msg.response_metadata.items():
    print(f"{key}: {value}")

# 如果模型调用了工具（function call）
if ai_msg.tool_calls:
    for call in ai_msg.tool_calls:
        tool_name = call['name']
        args = call['args']

        print("触发工具调用：", tool_name, args)

        # 5. 执行工具函数
        result = get_weather.run(args)

        # 6. 把工具执行结果作为 ToolMessage 传回模型
        final_msg = llm.invoke(
            [
                user_msg,
                ai_msg,
                ToolMessage(content=json.dumps(result), tool_call_id=call['id'])
            ]
        )
        print("\n最终自然语言回答:", final_msg.content)
